### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

### LINEAR REGRESSION

In [26]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL
    linear = LinearRegression()
    
    # TRAIN WITH TRAIN DATA
    model = linear.fit(data['train']['features'], data['train']['labels'])
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    # REVERSE NORMALIZATION FROM PREDICTIONS
    real_predictions = data['scaler'].inverse_transform(predictions)
    
    # ValueError: Expected 2D array, got 1D array instead:
    # Reshape your data either using array.reshape(-1, 1) if your
    # data has a single feature or array.reshape(1, -1) if it contains a single sample.
    
    return {
        'model': model,
        'predictions': real_predictions
    }

### LONG-SHORT TERM MEMORY

In [9]:
def lstm_add(model, data, index, name, settings):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }
    
    # SELECT THE CORRECT FUNCTION
    func = available[name]
    
    # IF AN ACTIVATION IS FOUND & THIS IS THE FIRST LAYER
    if 'activation' in settings and index == 0:
        model.add(func(
            settings['value'],
            activation=settings['activation'],
            input_shape=(data['train']['features'].shape[1], 1)
        ))
    
    # JUST AN ACTIVATION FUNCTION
    elif 'activation' in settings:
        model.add(func(
            settings['value'],
            activation=settings['activation']
        ))
        
    # OTHERWISE, DEFAULT TO JUST USING THE VALUE
    else:
        model.add(func(
            settings['value']
        ))

In [16]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL
    model = Sequential()
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # GENERATE & ADD THE LAYER
        lstm_add(model, data, index, name, params)
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    # TRAIN USING TRAIN DATA
    model.fit(
        data['train']['features'],
        data['train']['labels'],
        epochs=settings['epochs'],
        batch_size=settings['batch'],
        
        # VALIDATION_SPLIT WORKS NORMALLY, BUT SUCKS FOR TIMESERIES
        # https://www.tensorflow.org/tutorials/structured_data/time_series
        # goal https://miro.medium.com/max/300/1*R09z6KNJuMkSmRcPApj9cQ.png
        
        # ADD VALIDATION DATA
        validation_data=(
            data['validation']['features'],
            data['validation']['labels']
        ),
        
        # VALIDATE EVERY 25 STEPS
        validation_steps=settings['validation']
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    # REVERSE NORMALIZATION FROM PREDICTIONS
    real_predictions = data['scaler'].inverse_transform([predictions])
    
    return {
        'model': model,
        'predictions': np.ndarray.flatten(real_predictions)
    }

### START TRAINING A MODEL

In [11]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)